In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [2]:
path = r'Z:\TeleBrain\convert_edf_annot_break'
labels = pd.read_csv(r"Z:\TeleBrain\dataset.csv", sep=';')

In [6]:
data = []
for filename, label in zip(os.listdir(path), labels['PD']):
    if filename.endswith('.edf'):
        filepath = os.path.join(path, filename)
        raw = mne.io.read_raw_edf(filepath, preload=True)
        raw = raw.pick_channels(['C3'])
        events = mne.make_fixed_length_events(raw, duration=1.0)
        raw.filter(1, 40, fir_design='firwin')
        epochs = mne.Epochs(raw, events, tmin=0, tmax=2, baseline=None)
        epo_spectrum = epochs.compute_psd(method='multitaper', fmin=5, fmax=30, picks='eeg')
        psds, freqs = epo_spectrum.get_data(return_freqs=True)
        for p in psds:
            psds = np.mean(p, axis=0)
            data.append([psds.tolist(), label])
        break

Extracting EDF parameters from Z:\TeleBrain\convert_edf_annot_break\PD129.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 755199  =      0.000 ...  1474.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (3.303 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Not setting metadata
1475 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1475 events and 1025 original time points ...
2 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows


In [27]:
X_train, X_test, y_train, y_test = train_test_split([x[0] for x in data], [x[1] for x in data], test_size=0.2, random_state=42)
X_train_2d = np.array([x[0] for x in X_train]).reshape(len(X_train), -1)
X_test_2d = np.array([x[0] for x in X_test]).reshape(len(X_test), -1)
y_train=np.array(y_train)
y_train=y_train.transpose()
y_test=np.array(y_test)
y_test=y_test.transpose()
svm = SVC(kernel='linear', C=1, random_state=42)

In [28]:
print(X_test_2d.shape)
print(X_test_2d)
print(X_train_2d.shape)

(27873, 1)
[[1.54825669e-09]
 [7.06384206e-09]
 [2.99407427e-09]
 ...
 [1.24921947e-10]
 [2.38274392e-09]
 [2.00718238e-09]]
(111490, 1)


In [29]:
svm.fit(X_train_2d, y_train)

SVC(C=1, kernel='linear', random_state=42)

In [31]:
y_pred = svm.predict(X_test_2d)

In [36]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.4382018440784989
